In [9]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [10]:
train = pd.read_csv("/Users/mac/Documents/MySchoolDocs/PERSONAL/JOB APPLICATIONS/ENERGY_PROJECT/data/final_datasets/train_df.csv")
eval = pd.read_csv("/Users/mac/Documents/MySchoolDocs/PERSONAL/JOB APPLICATIONS/ENERGY_PROJECT/data/final_datasets/eval_df.csv")

In [11]:
X_train = train.drop(columns=["Date", "Strain_Index", "target_strain_index"])
y_train = train["target_strain_index"]

X_test = eval.drop(columns=["Date", "Strain_Index", "target_strain_index"])
y_test = eval["target_strain_index"]

In [12]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [13]:
params = dict(
    objective="regression",
    metric="mae",          # track MAE during training
    n_estimators=5000,     # rely on early stopping
    learning_rate=0.03,
    num_leaves=63,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    verbose=-1
)

lgbm = lgb.LGBMRegressor(**params)

In [14]:
lgbm.fit(
    X_train, y_train
)

y_pred_lgb = lgbm.predict(X_test)

In [16]:
def mape(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None))) * 100

print("LightGBM on Strain Index (t+1)")
print("--------------------------------")
print(f"MAE  : {mean_absolute_error(y_test, y_pred_lgb):.4f}")
print(f"RMSE : {root_mean_squared_error(y_test, y_pred_lgb):.4f}")
print(f"MAPE : {mape(y_test, y_pred_lgb):.2f}%")

LightGBM on Strain Index (t+1)
--------------------------------
MAE  : 0.0411
RMSE : 0.0477
MAPE : 51.00%
